In [1]:
import numpy as np
import cv2
import time
import torch
from Kalman_filter import Kalman_filter
from utilities import bb_iou, association_metric
from Tracker import Tracker
import warnings
warnings.filterwarnings("ignore")

In [2]:
#import the detection model with coco weights
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
person_id = 0

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-11-16 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
# read the video
capture = cv2.VideoCapture('input.mp4')
# Configure settings to save the output video
fourcc  = cv2.VideoWriter_fourcc(*'MP4V')
width, height = capture.get(cv2.CAP_PROP_FRAME_WIDTH), capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = capture.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter('output.mp4', fourcc, fps, (int(width), int(height)))

tracking = Tracker()
while capture.isOpened():
  try:
      bool_result,frame_feed = capture.read()
      results = model(frame_feed)
      results = results.xyxy[0].cpu().numpy()
      results = results[results[:,-2] > 0.5]
      results = results.astype(int)
      results = results[results[:, -1] == person_id]
      tracking.predict()
      un = tracking.update(results[:,:4])
      for i, trk in enumerate(tracking.tracks):
        if i not in un:
          cv2.rectangle(frame_feed, (int(trk.X[0])+2,int(trk.X[1])),(int(trk.X[2])+2, int(trk.X[3])), (255,0,0) ,2)  # filled
          cv2.putText(frame_feed,str(trk.track_id),(int(trk.X[0])+2,int(trk.X[1])),cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),2)
      out.write(frame_feed)
  except:
    break
out.release()